# 手刻神經網絡 with numpy

numpy.random.seed(number) → 控制output 結果  
numpy.random.random((size=None)) → 建立一個數值為0到1之間的隨機array  
numpy.zeros((shape)) → 建立一個全部為0的矩陣  
numpy.matmul(matrix1, matrix2) → 矩陣相乘  
numpy.vstack(matrix1, matrix2) → 已列的方式對array或matrix做組合  

In [62]:
import numpy as np

# 1. Derivative 微分

###  (1)先寫出一個簡單的方程式：  $ f(x) = 0.05x^2 + 0.8x $

In [63]:
def my_function(x):
    y = 0.05*x**2 + 0.8*x
    return y

### (2)定義微分公式：$ \frac{df(x)}{dx} =\lim\limits_{h\to 0} \frac{(f(x+h) - f(x))}{h} $
> 提示：不要將 $ epsilon $設的太大或是太小

In [64]:
# Define derivative, f是一個函數
epsilon = 0.1
def derivative(f, x, epsilon=0.1):
    # f is our function, and x is our variable
    h = epsilon
    return (f(x+h) - f(x)) / h

### (3)用我們的公式算 $x=5$ 和 $x=10$ 的微分
>可以嘗試改變epsilon來觀察微分的變化

In [65]:
print(derivative(my_function,5))
print(derivative(my_function,10))
print(derivative(my_function,5,1e-50))
print(derivative(my_function,10,1e-50))
print(derivative(my_function,5,1e-4))
print(derivative(my_function,10,1e-4))
print(derivative(my_function,5,1))
print(derivative(my_function,10,1))

1.3049999999999962
1.8050000000000033
0.0
0.0
1.3000049999956076
1.800005000003324
1.3500000000000005
1.8500000000000014


### 2. Partial Derivative  
###  (1)先寫出一個簡單的方程式：  $ f(x) = 2x^2 + 3xy + 5y^2 $

In [66]:
#X=(x,y), X[0] = x, X[1]=y
def my_function2(X):
    # xlist is the list of the variables we have
    # and in this particular instance we have two variables  
    return 2*X[0]**2+3*X[0]*X[1]+5*X[1]**2

## (2)定義偏微分公式：$ \frac{df(x_i)}{dx_a} =\lim\limits_{h\to 0} \frac{f(x_1,\,\cdots\,x_a\,+\,h,\,x_{a+1},\,\cdots\,x_n)\,-\,f(x_1\,\cdots\,x_n)}{h} $
>提示：可以嘗試建立一個新的array, 然後把xlist裡面第i個位置的值update

In [67]:
epsilon = 1e-6
def partial_derivative(f, X, i,epsilon=1e-6):
    # f is our function, and i is simply the index which we are 
    # excuting our partial derivative    
    H = X.copy()
    h = epsilon
    H[i] = X[i] + h
    return (f(H) - f(X)) / h    

### (3)用我們的公式算 $f_x(2,\,3)$ 和 $f_y(2,\,3)$ 的微分
>可以重新定義一個function來看partial對於你自己寫的function的變化<br>
>Your answer should be around 17 and 36

In [68]:
print(partial_derivative(my_function2, np.array([2., 3.]), 0,epsilon=1e-6))
print(partial_derivative(my_function2, np.array([2., 3.]), 1,epsilon=1e-6))

17.000002003442205
36.000004996594726


接下來讓我們自己手寫看看 $gradient$ 的程式，以下提供 $gradient$ 的算式
### $ \nabla f({x_1},{x_2},{x_3},\,\ldots) = \frac{\partial f}{\partial {x_1}}i+ \frac{\partial f}{\partial {x_2}}j+ \frac{\partial f}{\partial {x_3}}k \cdots$
<hr>

> 提示：可以使用 $for\,loop$ 來寫

In [69]:
def gradient(f, X):
    grad = []
    for i in range(len(X)):
        grad.append(partial_derivative(f, X, i))
    return grad

In [70]:
gradient(my_function2,np.array([2., 3.]))

[17.000002003442205, 36.000004996594726]

### 4. Loss 
>Definition: a loss function or cost function is a function that maps an event or values <br> 
>of one or more variables onto a real number intuitively representing some "cost" <br>
>associated with the event. <br>  

常用的$loss:\,Mean\,Square\,Error\,(MSE)$ <br>
### $MSE\,=\,\frac{1}{n} \displaystyle\sum_{i=1}^{n} (Y_i-\hat{Y_i})^2$

In [71]:
def MSE(true, pred):
    MSE = 0
    for i in range(len(true)):
        MSE += (true[i]-pred[i])**2
    return MSE/len(true)

### dataset

In [72]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
Y = iris.target
Y = Y.reshape(len(Y), 1)
names = iris.target_names

# Train valid test split

X_train = np.vstack([X[0:40], X[50:90], X[100:140]])
X_valid = np.vstack([X[40:45], X[90:95], X[140:145]])
X_test = np.vstack([X[45:50], X[95:100], X[145:150]])

Y_train = np.vstack([Y[0:40], Y[50:90], Y[100:140]])
Y_valid = np.vstack([Y[40:45], Y[90:95], Y[140:145]])
Y_test = np.vstack([Y[45:50], Y[95:100], Y[145:150]])

### (1)定義loss function  
將我們所有的資料X乘上我們的Weight W (我們這邊先不管Bias)  
然後我們使用的是MSE  
所以我們要套用MSE的算式如以下  

### $f\,=\,\frac{1}{2n} \displaystyle\sum_{i=1}^{n} (Y_i-XW)^2$


In [73]:
def function(W, data=X_train, target=Y_train):
    #矩陣相乘
    Z = np.matmul(data, W)
    f = MSE(target, Z)/2
    return f

### gradient_descent  

In [74]:
def gradient_descent(X_train, Y_train, X_valid, Y_valid, W, alpha, num_iters):
    m = len(Y_train)
    train_loss = np.zeros((num_iters, 1))
    valid_loss = np.zeros((num_iters, 1))
    
    for i in range(num_iters):
        # function代W求維分
        delta = gradient(function, W)
        delta = np.array(delta)
        W -= alpha*delta
        
        train_loss[i] = MSE(Y_train, np.matmul(X_train, W))
        valid_loss[i] = MSE(Y_valid, np.matmul(X_valid, W))
        
        if i%10 == 0:
            print('The training loss of the', i+1, 'epoch is', train_loss[i][0].round(4), 
                  ', and the validation loss of the', i+1, 'epoch is', valid_loss[i][0].round(4))
    return W, train_loss, valid_loss

# Initializing 
np.random.seed(37)
W = np.random.random((4,1))
W, train_loss, valid_loss = gradient_descent(X_train, Y_train, X_valid, Y_valid, W, 0.03, 100)

The training loss of the 1 epoch is 42.1148 , and the validation loss of the 1 epoch is 39.4722
The training loss of the 11 epoch is 2.715 , and the validation loss of the 11 epoch is 2.4563
The training loss of the 21 epoch is 0.2498 , and the validation loss of the 21 epoch is 0.1886
The training loss of the 31 epoch is 0.0764 , and the validation loss of the 31 epoch is 0.0403
The training loss of the 41 epoch is 0.0593 , and the validation loss of the 41 epoch is 0.027
The training loss of the 51 epoch is 0.0565 , and the validation loss of the 51 epoch is 0.0244
The training loss of the 61 epoch is 0.0557 , and the validation loss of the 61 epoch is 0.0234
The training loss of the 71 epoch is 0.0554 , and the validation loss of the 71 epoch is 0.0229
The training loss of the 81 epoch is 0.0553 , and the validation loss of the 81 epoch is 0.0227
The training loss of the 91 epoch is 0.0551 , and the validation loss of the 91 epoch is 0.0225


In [75]:
print(W)
predict = np.matmul(X_test, W).round()
print('The MSE score of our prediction is ', MSE(Y_test, predict)[0])

[[ 0.04607155]
 [-0.17754254]
 [ 0.13631098]
 [ 0.63300481]]
The MSE score of our prediction is  0.0


## MLP
<hr>
Sigmoid:  
Sigmoid (AKA logistic regression) 是一個用來把資料壓到 0 跟 1 之間的一個 function<br>
雖然 Sigmoid 非常好用 <br>
不過之後的課程也會提到 Sigmoid 的一些壞處<br>
請各位另用以下的算式寫出一個 Sigmoid 的 function<br>


### $ Z(x_i) = \frac{1}{1+e^{-x_i}}$
<hr>

>提示：可以使用 np.exp 來算 e

In [76]:
# define activation: sigmoid
def sigmoid(X):
    output = 1 / (1 + np.exp(-X)) 
    return output

# Examples
X = np.arange(5)
sigmoid(X)

array([0.5       , 0.73105858, 0.88079708, 0.95257413, 0.98201379])

## Sigmoid Gradient: 
sigmoid gradient的式子<br>
### $ \frac{d}{dx} Z(x_i) = Z(x_i) \times (1-Z(x_i))$  
[數學推導過程](http://www.ai.mit.edu/courses/6.892/lecture8-html/sld015.htm)


In [77]:
def sigmoid_gradient(X):
    output = sigmoid(X)*(1-sigmoid(X))
    return output

## Softmax:
<hr>
Softmax (AKA normalized exponential function) 與 Sigmoid 有點類似 <br>
不過 Softmax 是針對一整組數字做壓縮 <br>
而 Sigmoid 是直接對單一的數值做壓縮
請各位另用以下的算式寫出一個 Softmax 的 function<br>


### $\sigma(\mathbf{x})_j = \frac{e^{x_j}}{\sum_{i=1}^K e^{x_i}}\,for\,j\,\in\,1,\,\ldots\,,\,K$ 

For example:<br>
如果我們有一組數列 $1,\,3,\,5$ <br>
Softmax 會回傳 $0.016,\,0.117,\,0.867$

<hr>

>提示：python 會自動 broadcast

In [78]:
# define activation: softmax
def softmax(X): 
    return np.exp(X) / np.sum(np.exp(X), axis=1, keepdims=True)

# Examples
X = np.array([np.arange(5)])
softmax(X)

array([[0.01165623, 0.03168492, 0.08612854, 0.23412166, 0.63640865]])

## Cross entropy (Multiclass)
<hr>
Definition: Cross-entropy loss, or log loss, measures the performance of a classification model<br>
whose output is a probability value between 0 and 1. Cross-entropy loss increases as the predicted<br>
probability diverges from the actual label. So predicting a probability of .012 when the actual<br>
observation label is 1 would be bad and result in a high loss value. A perfect model would have a <br>
log loss of 0.<br><br>
Cross entropy 的算式如下：<br>

### $H(p,q)=-\displaystyle\sum _{i=1}^m  p(x_i)\,\log q(x_i).\!$
在這邊 p 代表的是我們的實際值，q 代表的是我們的預測值，$x_i$ 是我們的samples，m 是sample總數。<br>
如果還是不太清楚的話以下提供一個例子： <br><br>
>假設我們的第一個Y值(實際值)為 \[1, 0, 0\]，預測值為 \[0.7, 0.2, 0.1\] 的話
>#### $p(x_{0})\,\log q(x_{0})=1\times \log 0.7 + 0\times \log 0.2 + 0\times \log 0.1 \approx -0.357$
<!---
1. 假設我們的實際值為0，預測值為0.1的話
#### $-\sum_{x_i}p(x_{i})\,\log q(x_{i})=0\times \log 0.1 +(1-0) \times \log (1-0.1) = 0.105$
--->
這邊會是負數是因為log 1 到 0 之間的數值的話都會回傳負值<br>
但是沒關係  <br>
我們做完sum之後會在乘上 -1  <br><br>
請大家用這個算式自己寫寫看一個cross entropy 的function <br>
<hr>

><b>提示：</b> <br>
>由於log(0)會出現錯誤，所以我們通常會加一個epsilon，通常會設定為1e-15

In [79]:
# p , q is list
def cross_entropy(p, q):
    epsilon = 1e-15
    H = 0
    for i in range(len(p)):
        # epsilon avoid 0
        H += -p[i]*np.log(q[i]+epsilon)       
    H = H.sum()/p.shape[0]
    return H

p = np.array([[1,0,0]])
q = np.array([[0.7, 0.2, 0.1]])
cross_entropy(p,q)

0.356674943938731

## One Hot Encoding
<hr>
One Hot Encoding 做的事情就是把一個有 N 個不同種類的 array<br>
變成一個有 N 個 Column 的矩陣<br>
然後每一個 Column 都只會出現 1 或是 0<br>
每一個 Column 也都對應到 N 個種類中的其中一種<br>

In [80]:
def one_hot_encoding(array):
    
    sorted_array = np.sort(array)
    count = 1
    unique = [sorted_array[0]]
    
    temp = sorted_array[0]
    for i in range(len(array)):
        if sorted_array[i] != temp:
            count += 1
            temp = sorted_array[i]
            unique.append(temp)
            
    eye = np.zeros((len(unique), len(unique)))
    for i in range(len(unique)):
        eye[i, i] = 1
    
    for i in range(len(array)):
        for j in range(len(unique)):
            if array[i] == unique[j]:
                array[i] = j
                break
                
    result = eye[array]
    
    return result

# Example.
array = np.array([1,2,3,2])
one_hot_encoding(array)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

神經網路架構<br>
流程是<br>
Forward
1. $layer_1 = X \times weight_1 $
2. $activation_1 = activation(z_1)$
3. $layer_2 = a_1 \times weight_2$
4. $prediction = activationo(z_2)$<br>


Loss Function<br>

1. $loss = loss\,function(true\,value,\,prediction)$<br>

Backward<br>

1. $derivative\,of\,layer_2 = prediction - true\,value$
2. $derivative\,of\,weight_2  = activation_1\times derivative\,of\,layer_2$
3. $derivative\,of\,layer_1 = derivative\,of\,layer_2\times weight_2\times gradient\,of\,activation_1$
4. $derivative\,of\,weight_1  = X\times derivative\,of\,layer_1$

兩件事情<br>
第一件事情是我們這邊沒有加上bias<br>
這邊沒有加上bias是因為會讓model變得比較複雜<br>
為了簡化model所以沒有加上去<br>
第二件事情是Backward的第一步<br>
為什麼沒有做 softmax 和 cross entropy 的微分<br>
然後就用 prediction 減掉 true value<br>
其實是因為這個就是 cross entropy 加上 softmax 後的微分<br>
[Beautiful Math](https://deepnotes.io/softmax-crossentropy)



In [81]:
def two_layer_net(X, Y, W1, W2):
    # Forward
    z1 = np.matmul(X, W1) 
    a1 = sigmoid(z1)
    z2 = np.matmul(a1, W2) 
    out = softmax(z2)
    J = cross_entropy(Y, out)
    # Backward
    d2 = out - Y
    dW2 = np.matmul(a1.T, d2)
    d1 = np.matmul(d2, (W2.T))*sigmoid_gradient(a1)
    dW1 = np.matmul(X.T, d1)
    
    return J, dW1, dW2

### Training Data
>提示：如果 Function output 三個東西可是我們只要一個東西的話<br>
>     我們可以用底線 _ 來忽略
```python 
J_valid, _, _ = two_layer_net......
``` 
><br>或是我們可以直接用中括弧跟數字來代表<br>
```python
J_valid = two_layer_net(......)[0]
```

In [83]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
Y = iris.target
Y = one_hot_encoding(Y)
names = iris.target_names

X_train = np.vstack([X[0:40], X[50:90], X[100:140]])
X_valid = np.vstack([X[40:45], X[90:95], X[140:145]])
X_test = np.vstack([X[45:50], X[95:100], X[145:150]])

Y_train = np.vstack([Y[0:40], Y[50:90], Y[100:140]])
Y_valid = np.vstack([Y[40:45], Y[90:95], Y[140:145]])
Y_test = np.vstack([Y[45:50], Y[95:100], Y[145:150]])
#----------------------------------------------------
iteration = 1000
alpha = 0.01
history_train = np.zeros((iteration, 1))
history_valid = np.zeros((iteration, 1))

np.random.seed(37)
W1 = np.random.randn(4,10)
W2 = np.random.randn(10,3)

for i in range(iteration):
    J_train, dW1, dW2 = two_layer_net(X_train, Y_train, W1, W2)
    J_valid, _, _ = two_layer_net(X_valid, Y_valid, W1, W2)
    W1 -= alpha*dW1
    #b1 -= alpha*db1
    W2 -= alpha*dW2
    #b2 -= alpha*db2
    
    history_train[i] = J_train
    history_valid[i] = J_valid
    if (i+1)%50 == 0:
        print('The training loss of the', i+1, 'epoch is', history_train[i][0].round(4), ', ', end='')
        print('The validation loss of the', i+1, 'epoch is', history_valid[i][0].round(4))
        
print('\nThe loss of our testing set is ', two_layer_net(X_test, Y_test, W1, W2)[0].round(4))

The training loss of the 50 epoch is 0.5166 , The validation loss of the 50 epoch is 0.5436
The training loss of the 100 epoch is 0.4769 , The validation loss of the 100 epoch is 0.5154
The training loss of the 150 epoch is 0.4492 , The validation loss of the 150 epoch is 0.492
The training loss of the 200 epoch is 0.3166 , The validation loss of the 200 epoch is 0.2497
The training loss of the 250 epoch is 0.244 , The validation loss of the 250 epoch is 0.206
The training loss of the 300 epoch is 0.1796 , The validation loss of the 300 epoch is 0.1811
The training loss of the 350 epoch is 0.119 , The validation loss of the 350 epoch is 0.1182
The training loss of the 400 epoch is 0.1348 , The validation loss of the 400 epoch is 0.0478
The training loss of the 450 epoch is 0.1349 , The validation loss of the 450 epoch is 0.0451
The training loss of the 500 epoch is 0.1336 , The validation loss of the 500 epoch is 0.0412
The training loss of the 550 epoch is 0.1308 , The validation loss